In [29]:
import plotly.graph_objs as go  # 載入plotly套件
import numpy as np              # 載入numpy套件
import pandas as pd             # 載入pandas套件
import datetime as dt           # 載入datetime套件
from datetime import datetime

In [30]:
profit_df = pd.read_csv('trading_records.csv',encoding = 'utf-8')

In [31]:
profit_df.head(5)

,日期,Profit,策略損益(金額)
0,2000-02-01 00:00:00+08:00,10000.0,0.0
1,2000-02-02 00:00:00+08:00,10000.0,0.0
2,2000-02-03 00:00:00+08:00,10000.0,0.0
3,2000-02-04 00:00:00+08:00,10000.0,0.0
4,2000-02-07 00:00:00+08:00,10000.0,0.0


In [32]:
# 初始資金10000
capital = 10000
# 初始化策略累積資金
profit_df.loc[0, '策略累積資金(金額)'] = capital

# 開始累積
for i in range(1, len(profit_df)):
    profit_df.loc[i, '策略累積資金(金額)'] = profit_df.loc[i-1, '策略累積資金(金額)']+profit_df.loc[i, '策略損益(金額)']

profit_df.head(5)

,日期,Profit,策略損益(金額),策略累積資金(金額)
0,2000-02-01 00:00:00+08:00,10000.0,0.0,10000.0
1,2000-02-02 00:00:00+08:00,10000.0,0.0,10000.0
2,2000-02-03 00:00:00+08:00,10000.0,0.0,10000.0
3,2000-02-04 00:00:00+08:00,10000.0,0.0,10000.0
4,2000-02-07 00:00:00+08:00,10000.0,0.0,10000.0


In [33]:
# 計算每日報酬率
profit_df['策略每日報酬率'] = profit_df['策略累積資金(金額)'].pct_change()
profit_df.loc[0, '策略每日報酬率'] = 0  # 初始化第一天策略每日報酬率

# 計算累積報酬率
profit_df['策略累積報酬率'] = (1 + profit_df['策略每日報酬率']).cumprod()-1
profit_df.loc[0, '策略累積報酬率'] = 0  # 初始化第一天策略累積報酬率

# 計算Drawdown
# 計算每個時間點的最大價格
profit_df['每個時點最大策略累積資金(金額)'] = profit_df['策略累積資金(金額)'].cummax()
# 計算每個時間點的 Drawdown
profit_df['Drawdown'] = (profit_df['策略累積資金(金額)'] - profit_df['每個時點最大策略累積資金(金額)']) / capital

profit_df

,日期,Profit,策略損益(金額),策略累積資金(金額),策略每日報酬率,策略累積報酬率,每個時點最大策略累積資金(金額),Drawdown
0,2000-02-01 00:00:00+08:00,10000.000000,0.000000,10000.000000,0.000000,0.000000,10000.000000,0.000000
1,2000-02-02 00:00:00+08:00,10000.000000,0.000000,10000.000000,0.000000,0.000000,10000.000000,0.000000
2,2000-02-03 00:00:00+08:00,10000.000000,0.000000,10000.000000,0.000000,0.000000,10000.000000,0.000000
3,2000-02-04 00:00:00+08:00,10000.000000,0.000000,10000.000000,0.000000,0.000000,10000.000000,0.000000
4,2000-02-07 00:00:00+08:00,10000.000000,0.000000,10000.000000,0.000000,0.000000,10000.000000,0.000000
...,...,...,...,...,...,...,...,...
5033,2024-03-12 00:00:00+08:00,13446.984749,-250.829346,13446.984749,-0.018312,0.344698,87306.517212,-7.385953
5034,2024-03-13 00:00:00+08:00,13307.635872,-139.348877,13307.635872,-0.010363,0.330764,87306.517212,-7.399888
5035,2024-03-14 00:00:00+08:00,14171.600960,863.965088,14171.600960,0.064923,0.417160,87306.517212,-7.313492
5036,2024-03-15 00:00:00+08:00,13765.600960,-406.000000,13765.600960,-0.028649,0.376560,87306.517212,-7.354092


In [34]:
from plotly.subplots import make_subplots  #載入plotly畫子圖的套件

# 定義繪製策略損益圖方法
def draw_PnL(trace1, trace2,  title1, title2):
    
    # 打開子畫布
    fig = make_subplots(rows=2, cols=1, subplot_titles=(title1, title2), row_heights=[0.75, 0.25], shared_xaxes=True, vertical_spacing=0.1)
    
    # 帶入資料到子畫布的特定位置
    fig.add_trace(trace1, row=1, col=1)
    fig.add_trace(trace2, row=2, col=1)
    
    # 設定子畫布共同屬性
    fig.update_layout(
        title='Bollinger Bands', # 設定圖表標題名稱
        plot_bgcolor='white', # 設定底色為白色
    )
    
    # 設定子畫布屬性
    fig.update_xaxes(row=1, col=1, linecolor='black', linewidth=2, showgrid=True, gridcolor='lightgrey')
    fig.update_yaxes(title_text="損益(%)", row=1, col=1, linecolor='black', linewidth=2, showgrid=True, gridcolor='lightgrey', zerolinecolor='darkgrey')
    fig.update_xaxes(title_text="日期", row=2, col=1, linecolor='black', linewidth=2, showgrid=True, gridcolor='lightgrey')
    fig.update_yaxes(title_text="損益(%)", row=2, col=1, linecolor='black', linewidth=2, showgrid=True, gridcolor='lightgrey', zerolinecolor='darkgrey')

    #展示圖表
    fig.show()
    
    return

In [35]:
import pandas as pd

# 將日期轉換為 datetime 物件
profit_df['日期'] = pd.to_datetime(profit_df['日期'])


**<H>特殊格式<H>**

In [36]:
# 建立plotly損益曲線
trace1 = go.Scatter(x=profit_df['日期'].dt.strftime('%Y-%m'),  # 格式化為年月
                    y=profit_df['策略累積報酬率']*100, 
                    mode='lines', 
                    name='策略損益曲線', 
                    line=dict(color='blue')
                    )
trace2 = go.Scatter(x=profit_df['日期'].dt.strftime('%Y-%m'),  # 格式化為年月
                    y=profit_df['Drawdown']*100, 
                    mode='lines', 
                    name='Drawdown', 
                    line=dict(color='red')
                    )
# 繪製策略損益圖
draw_PnL(trace1, trace2, '布林通道損益', 'Drawdown')

**<H>一般格式<H>**

In [37]:
# 建立plotly損益曲線
trace1 = go.Scatter(x=profit_df['日期'], 
                    y=profit_df['策略累積報酬率']*100, 
                    mode='lines', 
                    name='策略損益曲線', 
                    line=dict(color='blue')
                    )
trace2 = go.Scatter(x=profit_df['日期'], 
                    y=profit_df['Drawdown']*100, 
                    mode='lines', 
                    name='Drawdown', 
                    line=dict(color='red')
                    )
# 繪製策略損益圖
draw_PnL(trace1, trace2, '布林通道損益', 'Drawdown')

In [38]:
# 計算績效指標
total_return = profit_df.loc[len(profit_df)-1, '策略累積報酬率']
annualized_return = (1+profit_df.loc[len(profit_df)-1, '策略累積報酬率'])**(4/len(profit_df))-1  # 年化報酬率
annualized_volatility = np.std(profit_df['策略每日報酬率'])*np.sqrt(4)  # 年化波動率
sharpe_ratio = annualized_return/annualized_volatility  # 年化夏普值
max_drawdown = profit_df['Drawdown'].min()  # 最大回撤(MDD)

print("總報酬率: {}%".format(round(total_return*100,2)))
print("年化報酬率: {}%".format(round(annualized_return*100,2)))
print("年化波動率: {}%".format(round(annualized_volatility*100,2)))
print("年化夏普值: {}".format(round(sharpe_ratio,2))) 
print("最大回撤(MDD): {}%".format(round(max_drawdown*100,2)))

總報酬率: 33.6%
年化報酬率: 0.02%
年化波動率: 4.39%
年化夏普值: 0.01
最大回撤(MDD): -743.17%
